# Описание проекта

Работем в интересах интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Нам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

**Описание данных**

- Name — название игры

- Platform — платформа

- Year_of_Release — год выпуска

- Genre — жанр игры

- NA_sales — продажи в Северной Америке (миллионы проданных копий)

- EU_sales — продажи в Европе (миллионы проданных копий)

- JP_sales — продажи в Японии (миллионы проданных копий)

- Other_sales — продажи в других странах (миллионы проданных копий)

- Critic_Score — оценка критиков (максимум 100)

- User_Score — оценка пользователей (максимум 10)

- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

*Данные за 2016 год могут быть неполными.*

## Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
import copy
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats as st
import numpy as np
import datetime
import statsmodels.formula.api as smf
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('/datasets/games.csv')
df.info()

In [ ]:
display(df.head(7))

Посмотрим долю пропусков от общего числа записей

In [ ]:
pd.DataFrame((df.isna().mean()*100))

**Промежуточный вывод:**

Итого у нас 11 столбцов и 16715 строк. Названия столбцов и жанры необходимо привести к нижнему регистру, а так же для некоторых полей нужно заменить форматы. 

К тому же, мы видим, что у столбцов 'Name' и 'Genre' имеется незначительное количество пропусков. Для некоторых игр не указан год релиза. Около половины записей имеют пропуски в оценке критиков и игроков.

## Подготовьте данные

Сперва заменим названия столбцов (приведем к нижнему регистру)

In [ ]:
df.columns = df.columns.str.lower()
df.info()

Так-с, посчитаем пропуски и преобразуем данные в нужные для дальнейшего исследования типы данных

In [ ]:
df.isna().sum()

Посмотрим уникальные значения в колоке "оценка критиков"

In [ ]:
df['critic_score'].unique()

Теперь посмотрим количество пропущенных значений среди оценок критиков

In [ ]:
df['critic_score'].isna().sum()

Примем для отсутвующих значений оценки критиков "-1" и преобразуем в тип int

In [ ]:
df['critic_score'] = df['critic_score'].fillna(-1)
df['critic_score'] = df['critic_score'].astype(int)
df['critic_score'].unique()

Взглянем на количество пропущенных значений годе выпуска

In [ ]:
df['year_of_release'].isna().sum()

Что там по уникальным значениям в годе выпуска?

In [ ]:
df['year_of_release'].unique()

Удалим игры с пропущенными признаками года выпуска

In [ ]:
df = df.dropna(subset = ['year_of_release']).reset_index(drop = True)
df.info()

Переведём столбец в формат datetime и вытащим год из колонки

In [ ]:
df['year_of_release'] = pd.to_datetime(df['year_of_release'], format='%Y')
df['year_of_release'] = df['year_of_release'].dt.year 

Посмотрим на уникальные значения в колоке "оценка пользователей"

In [ ]:
df['user_score'].unique()

In [ ]:
df[df['user_score'] == 'tbd'].groupby(
    ['year_of_release','platform','name']).agg(
    {'user_score':'count'}).sort_values(by='user_score', ascending=False).head(10)

Получается что "tbd" означает *to be determined* - недостаточно пользовательских оценок для формирования рейтинга. Заменим "tbd" на "-2", а "nan" заменим на "-1", чтобы отличать эти значения. Поле "рейтиг" переведем из 10 бальной в 100 бальную шкалу, а так же заменим тип на целочисленные значения

In [ ]:
df['user_score'] = df['user_score'].replace("tbd", -0.2)
df['user_score'] = df['user_score'].fillna(-0.1)
df['user_score'] = df['user_score'].astype(float)
df['user_score'] = df['user_score']*10
df['user_score'] = df['user_score'].astype(int)
df['user_score'].unique()

In [ ]:
df['rating'].unique()

Присвоим значению "nan" более лаконичное "unknown"

In [ ]:
df['rating'] = df['rating'].fillna('unknown')

In [ ]:
df[df['rating'] == 'unknown'].groupby(
    ['na_sales','eu_sales','other_sales', 'jp_sales']).agg(
    {'user_score':'count'}).sort_values(by='user_score', ascending=False).head(20)

Больше всего пропусков в рейтинге у игр которые продавались за пределами Севенй Америки. Обратимся к Марксу(т.е. к Гуглу) и выясним, что:

Entertainment Software Rating Board (ESRB) — негосударственная организация, основное направление деятельности — принятие и определение рейтингов для компьютерных видеоигр и другого развлекательного программного обеспечения в США и Канаде.

Теперь всё становится достаточно логичным

Посмотрим строки без названия игры и удалим их, т.к. они не имеют значения для дальнейшего исследования

In [ ]:
df[df['name'].isnull()]

In [ ]:
df = df.dropna()

**Добавим столобец с просуммированными продажами по миру**

In [ ]:
df['word_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

In [ ]:
df['platform'].unique()

In [ ]:
df['genre'].unique()

На последок посмотрим наличие явных дубликатов

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(subset=['name', 'platform', 'year_of_release'], keep=False)]

Что касается Need for Speed: Most Wanted, то это две части игры, вышедшие в 2005 и 2012 году, просто под одним и тем же названием. Их мы оставляем.

А с дубликатами в строках с Madden NFL 13, чтобы не потерять точные продажи, добавим в первую строку в колонку "eu_sales" недостающие значения из дублирующей строки и удалим дубль.

In [ ]:
df.loc[df['name'] == 'Madden NFL 13', 'eu_sales'] = 0.23

df.drop(labels=[15971], axis=0, inplace=True)

In [ ]:
df[df.duplicated(subset=['name', 'platform'], keep=False)]

**Промежуточный вывыод:**

- Рейтинг критиков и пользователей привели к целым числам и сто бальной шкале
- Заменили пропуски в "годах выпуска" и "возрастной категории", где это было возможно
- Добавили колонку с суммой продаж по всему миру - 'word_sales' (в млн копий)
- Удалили оставшиеся пропуски в названиях
- Проверили датасет на явные дубликаты
- Пропуски в отзывах могли появиться, потому что продажи игры были низкие и не набралось необходимого количества индивидуальных отзывов для получения итоговой оценки

In [ ]:
# Комментарий ревьюера
# Посмотрим, что осталось
temp = df.copy() 
list_c = ['name', 'platform', 'year_of_release', 'genre', 'critic_score', 'user_score', 'rating']
print(temp.info())
for col_l in list_c:
  print('-'* 25)
  print(col_l, temp[col_l].sort_values().unique())
  print(col_l,': кол-во NaN',temp[col_l].isna().sum(),
        ', процент NaN', round(temp[col_l].isna().mean()*100,2),'%') 

## Проведите исследовательский анализ данных

### Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?

Проверим наличие неявных дубликатов наименованиях игр и их выпуске по годам

In [ ]:
df[df.duplicated(subset = ['name', 'year_of_release'], keep = False)].head(7)

Удалим дубли, за исключением первой строки, и сохраним в переменную

In [ ]:
unique_games = df.drop_duplicates(subset = ['name', 'year_of_release'], keep = 'first').reset_index(drop = True)

Создадим сводную таблицу с количеством игр в каждом году, будем считать по именам, т.к. они уникальны. После чего создадим график по сводной таблице

In [ ]:
pivot_games_by_year = unique_games.pivot_table(index = 'year_of_release', values = 'name', aggfunc = 'count')

fig, ax = plt.subplots()
sns.barplot(x = pivot_games_by_year.index,
            y = pivot_games_by_year.name)
plt.grid()
plt.title('Количество выпущенных игр на разных платформах по годам', fontsize = 25)
plt.xlabel('Год выпуска', fontsize = 20)
plt.ylabel('Количество новых игр', fontsize = 20)
plt.xticks(rotation = 35)
fig.set_figwidth(20)
fig.set_figheight(10);

**Промежуточные выводы:**

- С 2000 года отмечается рост количества новых игр, в 2008 году количество достигает своего пика (более 1000 игр), а с 2009 начинается спад. Возьмем в исследуемый диапазон игры, выпущенные с 2001 по 2016 годы.
- Такой спад можно охарактеризовать отсутствием новизны в сфере игр, всевозможные жанры и сюжеты были выпущены ранее, и геймдевам приходится заимствовать многое из старых игр, в попытках добавить что-то оригинальное
- Такую динамику также можно связать с появлением онлайн игр. Многие пользователи могут годами играть в одну и ту же онлайн-игру, не интересуясь новыми играми. Из-за этого спрос падает

### Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?

Для наглядности результатов дальнейших исследований отсеем неинформативные годы (т.е. период с 1980 по 2000)

In [ ]:
data_after_2000 = df.query('2000 <= year_of_release')

In [ ]:
data_after_2000.pivot_table(index = "year_of_release", columns = "platform", 
                 values = "word_sales", aggfunc = "sum").plot.bar(stacked = True, figsize = (20, 10), grid = True)
plt.title('Количество проданных игр на разных платформах по годам', fontsize = 25)
plt.xlabel('Год выпуска', fontsize = 20)
plt.ylabel('Продажи', fontsize = 20)
plt.show()

**Промежуточные выводы:**

- Новые платформы повяются примерно каждые пять лет
- Старые платформы изчезают примерно через два года после появления новых

### Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год. Не учитывайте в работе данные за предыдущие годы. Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.

выделим актуальный период

In [ ]:
df_after_2013 = df.query('year_of_release > 2011 and year_of_release < 2017')

построим график распределения по годам

In [ ]:
(
df_after_2013
    .pivot_table(index = 'year_of_release', columns = 'platform', values = 'word_sales', aggfunc = 'sum')
    .plot(
        grid = True,
        figsize = (15, 7),
        linewidth = 3
    )
)
plt.title('Распределение суммарных продаж по платформам за 2013-2016 годы', fontsize = 20)
plt.xlabel('Год выпуска', fontsize = 17)
plt.ylabel('Суммарный объем продаж', fontsize = 17)
plt.xticks(np.arange(2013, 2017, 1))
plt.yticks(np.arange(0, 150, 10));

In [ ]:
df_after_2013.pivot_table(index = "year_of_release", 
columns = "platform", values = "word_sales", aggfunc = "sum")

**Промежуточные выводы:**

- За период с 2013 по 2016 годы можно отметить две лидирующие платформы: PS4 и XOne. Для 2017 года они кажутся потенциально прибыльными. Так как игры на этих платформах появились в 2013 году, а средний срок актуальности платформы 8 лет – они должны просуществовать еще примерно 4 года
- PSP ушла с рынка в 2015 году
- Nintendo WII к 2016 году была уже с нулевыми продажами
- У остальных платформ значительно сокращаются объемы продаж, для 2017 года они будут уже не выгодными

Таким образом наиболее прибыльными за нужный нам период являются "PS4" и "XOne"

### Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.

In [ ]:
sns.boxplot(x = df_after_2013["word_sales"], y = df_after_2013["platform"], 
           order=["PS4", "PS3", "XOne",  "X360", "3DS", "WiiU", 'Wii']);
plt.xlim(0,3) 
plt.title('Глобальные продажи в разбивке по платформам', fontsize = 20)
plt.xlabel('Продажи', fontsize = 17)
plt.ylabel('Платформы', fontsize = 17)
plt.show()

**Промежуточные выводы:**

- У PS3 и PS4 одинаковые медиальные значения. Продажи им сделали 4 - 6 игр.
- X360 аналогичная ситуация, но среднее медиальное больше
- XOne менее всего выбросов. Более равномерное распределение и медеальное чуть меньше X360
- 3 DS имеет несколько игр которые сделали продажи платформе. Притом довольно малое медиальное значение
- У WiiU и Wii есть несколько игр, за счет чего выбивается из средних показателей

### Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы. Соотнесите выводы с продажами игр на других платформах.

Для точности в исследовании данного вопроса, уберём из выборки данные, у которых оценки критиков и пользователей имеют значения *-1* и *-2*, которые мы проставили вместо *tbd* и *nun* во время предобработки

In [ ]:
df_nnull = df_after_2013[df_after_2013['user_score'] > 0]
df_nnull = df_nnull[df_nnull['critic_score'] > 0]
df_nnull_user = df_after_2013[df_after_2013['user_score'] > 0]
df_nnull_critic = df_after_2013[df_after_2013['critic_score'] > 0]

In [ ]:
# Комментарий ревьюера
print(len(df_after_2013[df_after_2013['user_score'] > 0]))
print(len(df_after_2013[df_after_2013['critic_score'] > 0]))
print(len(df_nnull))

#### Теперь посмотрим как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков

Рассмотрим продажи игр для PS4 в связи с оценками критиков

In [ ]:
ps4_critic = df_nnull_critic.query('platform == "PS4"')
ps4_user = df_nnull_user.query('platform == "PS4"')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(x = ps4_critic['critic_score'], y = ps4_critic['word_sales'], alpha=0.3)
plt.xlabel("Оценки критиков")
plt.ylabel("Общие продажи")
fig.suptitle('PS4', fontsize=15)
plt.show()

In [ ]:
ps4_critic['critic_score'].corr(ps4_critic['word_sales'])

Как видим, между оценками критиков и общими продажами присутсвует небольшая положительная корреляция. Полагаю, что это закономерно, т.к. критики оценивают игру довольно углубленно, а значит анализируют больше моментов, на которые могут обратить внимание игроки при желании или нежелании покупать/проходить конкретную игру

А теперь взглянем на продажи игр для PS4 в связи с оценками пользователей

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(x = ps4_user['user_score'], y = ps4_user['word_sales'], alpha=0.3)
plt.xlabel("Оценки пользователей")
plt.ylabel("Общие продажи")
fig.suptitle('PS4', fontsize=15)
plt.show()

In [ ]:
ps4_user['user_score'].corr(ps4_user['word_sales'])

Коэфициент корреляции между оценками пользователей и общими продажами заметно меньше и имеет отрицательное значение. То есть по мере увеличения значений в столбце оценок пользователей значения в столбце общих продаж имеют тенденцию к уменьшению

#### Соотнесём выводы с продажами игр на других платформах.

Сначала посмотрим показания для платформы XOne. А уже затем нарисуем более общую конву графиков

In [ ]:
XOne_critic = df_nnull_critic.query('platform == "XOne"')
XOne_user = df_nnull_user.query('platform == "XOne"')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(x = XOne_critic['critic_score'], y = XOne_critic['word_sales'], alpha=0.3)
plt.xlabel("Оценки критиков")
plt.ylabel("Общие продажи")
fig.suptitle('XOne', fontsize=15)
plt.show()

In [ ]:
XOne_critic['critic_score'].corr(XOne_critic['word_sales'])

Как видим, корреляция, как и у PS4, между оценками критиков и общими продажами небольшая, но имеется

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(x = XOne_user['user_score'], y = XOne_user['word_sales'], alpha=0.3)
plt.xlabel("Оценки пользователей")
plt.ylabel("Общие продажи")
fig.suptitle('XOne', fontsize=15)
plt.show()

In [ ]:
XOne_user['user_score'].corr(XOne_user['word_sales'])

Аналогичная с PS4 картина

**Теперь посмотрим картину сравнения на освновании 5 популярных платформ**

*Сначала соотношение с пользовательскими оценками*

In [ ]:
for i in ['PS3', 'PS4', 'XOne', 'X360', '3DS']:
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.scatter(x = df_nnull_user.query('platform == @i')['user_score'], y = df_nnull_user.query('platform == @i')['word_sales'], alpha=0.3)
    plt.xlabel("Оценки пользователей")
    plt.ylabel("Общие продажи")
    fig.suptitle(i, fontsize=15)
    plt.show()
    print('Коэффициент корреляции', df_nnull_user.query('platform == @i')['user_score'].corr(df_nnull_user.query('platform == @i')['word_sales']))

**Промежуточный вывод:**

Сумма продаж практически всегда напрямую не связана с оценками пользователей

*Теперь соотношение с оценками критиков*

In [ ]:
 for i in ['PS3', 'PS4', 'XOne', 'X360', '3DS']:
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.scatter(x = df_nnull_critic.query('platform == @i')['critic_score'], y = df_nnull_critic.query('platform == @i')['word_sales'], alpha=0.3)
    plt.xlabel("Оценки критиков")
    plt.ylabel("Общие продажи")
    fig.suptitle(i, fontsize=15)
    plt.show()
    print('Коэффициент корреляции', df_nnull_critic.query('platform == @i')['critic_score'].corr(df_nnull_critic.query('platform == @i')['word_sales']))

**Промежуточный вывод:**

С увеличением среднего рейтинга критиков увеличивается и средний уровень выручки с продажи игр. Опять же, логика прослеживаемая: чем качественнее игра, тем больше денег она приносит разработчику.

**Промежуточные выводы по блоку с корреляцией:**

- На примере отдельно взятой платформы, мы выяснили, что оценки игроков почти не влияют на продажи игр. В общем то оценки пользователей как праило субъективны. Более того они выставляются по мере прохождения игры, в то время как основная часть покупок, как правило производится в даты близки к моменту релиза(когда количество пользовательских оценок не велико)
- В то же время оценки критиков не слишком сильно, но влияют на продажи. Этому могут послужить такие факторы, как: 
    - более компетентный и объективный взгляд критика
    - отзывы критиков как правило появляются в первые же дни релиза игры(или даже раньше), а значит хорошие отзывы критиков могут, и используются даже для продвижения самой игры, зачастую являясь ещё одним маркетинговым рычагом для компании-разработкика
- В сравнении между играми на разные платформы прослеживаются те же тенденции, что и на изначально отдельно взятой платформе. То есть оценки критиков скорее влияют на продажи, а оценки пользователей скорее нет.

### Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
plt.figure(figsize = (17,8))
sns.heatmap(df_after_2013.pivot_table(index = 'year_of_release', columns = 'genre', values = 'name', aggfunc = 'count'))
plt.title('Число игр по году релиза и жанру', fontsize = 14)
plt.xlabel('Жанр игры', fontsize = 14)
plt.ylabel('Год релиза игры', fontsize = 14)
plt.show()

In [ ]:
plt.figure(figsize = (17,8))
sns.heatmap(df_after_2013.pivot_table(index = 'year_of_release', columns='genre', values = 'word_sales', aggfunc = 'sum'))
plt.title('Продажи по году релиза и жанру', fontsize = 14)
plt.xlabel('Жанр игры', fontsize = 14)
plt.ylabel('Год релиза игры', fontsize = 14)
plt.show()

Посмотрим сводную таблицу по количеству игр каждого жанра

In [ ]:
pivot_genre = (df_after_2013
               .pivot_table(index = 'genre', values = 'name', aggfunc = 'count')
               .sort_values('name', ascending=False)
              )
pivot_genre

**Промежуточные выводы:**

- По количеству игр выделяются жанр экшн, который содержит более 700 игр
- Менее 50 игр в категории паззлы
- Остальные жанры находятся в промежутке от 50 до 300 игр

Посмотрим сводную таблицу по продажам игр каждого жанра

In [ ]:
pivot_genre_sales = (df_after_2013
               .pivot_table(index = 'genre', values = 'word_sales', aggfunc = 'median')
               .sort_values('word_sales', ascending = False)
              )
pivot_genre_sales

**Промежуточные выводы:**

- В топ-3 продаваемых жанра входят шутер, спортивные игры и платформеры
- Игр жанра шутер примерно 190, но по продажам он на первом месте, поэтому это перспективный жанр
- Жанр экшн, несмотря на такое количество игр, находится всего лишь на 7 месте по продажам
- В самом низу продаж три жанра: приключения, паззлы и стратегии

**Общие выводы по исследовательскому анализу данных:**

- С 2001 года начался значительный рост количества новых игр, а с 2010 отмечается резкий спад
- Примерный срок актуальности платформы 5 лет
- Лидирующими платформами являются PS4 и XOne, они появились в 2013 году, поэтому отсчет начинаем с 2013 года
- При росте оценки - количество продаж увеличивается, особенно при оценках критиков
- В топ-3 продаваемых жанра входят шутер, спортивные игры и платформеры
- Шутеры, несмотря на небольшое количество игр, являются перспективным жанром для продаж, а паззлы наоборот, самый малочисленный не прибыльный жанр

## Составьте портрет пользователя каждого региона. (Ошибочный блок. Далее сделаем данный раздел по новой под номером пункта 5)

### Определите для пользователя каждого региона (NA, EU, JP) самые популярные платформы (топ-5). Опишите различия в долях продаж.

In [ ]:
countries = df_after_2013.columns[4:7]
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Топ-5 платформ в регионах')
for i in range(3):
  df_i = (df_after_2013.groupby('platform')[countries[i]].sum().
             reset_index().
             sort_values(countries[i], ascending = False, ignore_index = True).
             head(5))
  df_i = df_i.append({'platform': 'Other_platforms', countries[i]: df[countries[i]].sum() - df_i[countries[i]].sum()}, ignore_index= True)
  axs[i].pie(df_i[countries[i]], labels = df_i['platform'])
  
  axs[i].set_title(df_i.columns[1].split('_')[0].upper())
plt.show()

**Промежуточные выводы:**

- В Северной Америке лидирует X360, однако две консоли разных поколений от Sony идут следом в не большом отрыве от лидера
- В Европе явно преобладают любители Sony. Наиборльший процент консольщиков из этого региона "гамают" на PS4
- В Японии 51% рынка занимает 3DS. Кажется любовь японцев к портативной игровой системе от Nintendo не смогли пошатнуть даже их сограждане из Sony

### Определите для пользователя каждого региона (NA, EU, JP) самые популярные жанры (топ-5). Поясните разницу.

In [ ]:
genres = df_after_2013.columns[4:7]
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Топ-5 жанров в регионах')
for i in range(3):
  df_i = (df_after_2013.groupby('genre')[genres[i]].sum().
             reset_index().
             sort_values(genres[i], ascending = False, ignore_index = True).
             head(5))
  df_i = df_i.append({'genre': 'Other_genres', genres[i]: df[genres[i]].sum() - df_i[genres[i]].sum()}, ignore_index= True)
  axs[i].pie(df_i[genres[i]], labels = df_i['genre'])

  axs[i].set_title(df_i.columns[1].split('_')[0].upper())
plt.show()

**Промежуточные выводы:**

- По жанрам в Северной Америке и Европе на вершинах популярности экшн, шутеры и спортивные игры. 
- Первое место по жанрам у Японии занимают ролевые игры. На втором уже столь любимый Американцами и Европейцами Экшн. А вот шутеры в топ-5 даже не вошли. Не потому ли, что в Японии очень строгое законадательство по огнестрельному оружию? Или культурный код диктует любовь к благородным катанам и вакидзаси, вместо трусливого огнестрела? Кто знает...

### Определите для пользователя каждого региона (NA, EU, JP), влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
ratings = df_after_2013.columns[4:7]
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Влияние рейтинга ESRB на продажи в регионах')
for i in range(3):
  df_i = (df_after_2013.groupby('rating')[ratings[i]].sum().
             reset_index().
             sort_values(ratings[i], ascending = False, ignore_index = True).
             head(5))
  df_i = df_i.append({'rating': 'Other_raitings', ratings[i]: df[ratings[i]].sum() - df_i[ratings[i]].sum()}, ignore_index= True)
  axs[i].pie(df_i[ratings[i]], labels = df_i['rating'])

  axs[i].set_title(df_i.columns[1].split('_')[0].upper())
plt.show()

Для понимания следующего показателя, нужно вообще вспомнить, а какие есть возрастные ограничения используемые ESRB:

✅ «E» («Everyone») — «Для всех»: Содержание вполне подходит для возрастной категории старше 6 лет; такие игры могут понравиться и взрослым.

✅ «EC» («Early childhood») — «Для детей младшего возраста»: Игра подходит для детей от 3 лет и старше и не содержит материалов, которые родители могли бы счесть неподходящими. Игры, получившие данный рейтинг, изначально разрабатываются для детей и обычно представляют собой развивающие игры.

✅ «E10+» («Everyone 10 and older») — «Для всех старше 10 лет»: Возрастная категория — от 10 лет. Проекты с данным рейтингом могут содержать немного больше мультипликационного или мягкого насилия, или несколько откровенные сцены.

✅ «T» («Teen») — «Подросткам 13—19 лет»: Игра подходит для подростков старше 13 лет. Проекты из данной категории могут содержать насилие, непристойные сцены, грубый юмор, не откровенное сексуальное содержимое или не частое использование ненормативной лексики.

✅ «M» («Mature») — «От 17 лет»: Материалы игры не подходят для возрастной категории младше 17 лет. Проекты с данным рейтингом могут содержать достаточно жестокое насилие, большое количество крови, непристойные сексуальные сцены или грубую ненормативную лексику.

✅ «E» («Everyone») - То бишь «Для детей и взрослых».

✅ «AO» («Adults Only 18+») — «Только для взрослых»: Содержание игры только для взрослых. Продукты из данной категории могут содержать длительные сцены жестокого насилия и/или очень откровенное сексуальное содержимое, а также сцены с обнажением.

**Промежуточные выводы:**

- В рейтингах у Северной Америки и Европы большую часть рынка занимает возрастная категория "M" — "От 17 лет". Видимо наибольшими потребителями игр там являются подростки и молодые люди.
- В Японии же первое место занимает категория «unknown». Это те самые 51% рынка игр для Nintendo, которые получают свои внутреннние оценки, и не нуждаются в оценке от ESRB. На втором по имеющимся данным, и первом по статистике ESRB месте, Японцы предпочитают игры категории "Е", то есть, для всех возрастов.

**Общие выводы по портретам пользователей в регионах:**

- По регионам самыми популярными платформами являются остаются PS4 и XOne(как мы уже видели при проведении исследовательского анализа). Правда, для японских пользователей нет ничего лучше чем игровые системы на платформе от Nintendo. Вот это я понимаю поддержка национального производителя.

- Самыми продаваемыми являются жанры - экшн, шутеры и ролевые игры.

- Идеальным пользователем для *Северной Америки*, является подросток или взрослый, играющий на платформах PS4 и XOne в экшн, шутеры и спортивные игры.

- Идеальный портрет *европейского* пользователя так же состоит из подростков и взрослых, играющих на платформе PS4 в экшн, шутеры и спортивные игры.

- Идеальный *японский* пользователь играет на консоли Nintendo в ролевые игры и экшн.

- В то же время трудно сказать, влияет ли оценка рейтинга от ESRB или других организаций на продажи в тех или иных регионах, т.к. не известно на сколько розничные торговцы в силах проконтролировать возраст конечного потребителя игры. Никто ведь не логинится в играх по паспорту, где можно точно узнать возраст пользователя. Хотя даже это не гарантирует, что залогинившийся пользователь, и тот кто непосредственно играет - один и тот же человек. Но оно, как мне кажется, и не нужно. В конце концов, я думаю, что объёмы продаж по возрастному рейтингу могут оценивать только создатели игр для более узкой аудитории(напрмер развивающие игры для детей от 3 лет)

## Составьте портрет пользователя каждого региона. (Новый блок)

### Определите для пользователя каждого региона (NA, EU, JP) самые популярные платформы (топ-5). Опишите различия в долях продаж.

Посмотрим на ситуацию с платформами в регионе NA

In [ ]:
(df_after_2013.groupby(['platform'])\
                  .agg({'na_sales':'sum'}).sort_values(by = 'na_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные платформы в регионе NA (в млн.)')
                    ) 
plt.show()

In [ ]:
na_platform = df_after_2013.pivot_table(index='platform', values='na_sales', aggfunc='sum').nlargest(5, 'na_sales')
all_na = df_after_2013.pivot_table(index='platform', values='na_sales', aggfunc='sum').nsmallest(5, 'na_sales')
all_na.loc['other_platform'] = all_na.sum()
na_platform = pd.concat([na_platform[:5], all_na[5:]])
na_platform

Посмотрим на ситуацию с платформами в регионе EU

In [ ]:
(df_after_2013.groupby(['platform'])\
                  .agg({'eu_sales':'sum'}).sort_values(by = 'eu_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные платформы в регионе EU (в млн.)')
                    ) 
plt.show()

In [ ]:
eu_platform = df_after_2013.pivot_table(index='platform', values='eu_sales', aggfunc='sum').nlargest(5, 'eu_sales')
all_eu = df_after_2013.pivot_table(index='platform', values='eu_sales', aggfunc='sum').nsmallest(5, 'eu_sales')
all_eu.loc['other_platform'] = all_eu.sum()
eu_platform = pd.concat([eu_platform[:5], all_eu[5:]])
eu_platform

Посмотрим на ситуацию с платформами в регионе JP

In [ ]:
(df_after_2013.groupby(['platform'])\
                  .agg({'jp_sales':'sum'}).sort_values(by = 'jp_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные платформы в регионе JP (в млн.)')
                    ) 
plt.show()

In [ ]:
jp_platform = df_after_2013.pivot_table(index='platform', values='jp_sales', aggfunc='sum').nlargest(5, 'jp_sales')
all_jp = df_after_2013.pivot_table(index='platform', values='jp_sales', aggfunc='sum').nsmallest(5, 'jp_sales')
all_jp.loc['other_platform'] = all_jp.sum()
jp_platform = pd.concat([jp_platform[:5], all_jp[5:]])
jp_platform

**Промежуточные выводы:**

- В Северной Америке лидирует X360, однако две консоли разных поколений от Sony идут следом в не большом отрыве от лидера.
- В Европе явно преобладают любители Sony. Наиборльший процент консольщиков из этого региона используют PS4 и PS3
- В Японии с заметным отрывом от конкуретнтов лидирует приставка 3DS. За ней следуют трио от Sony: PS3, PSV и PS4. Стоит отметить, что из топ-5 игровых консолей, 3 являются портативными, т.е. японцы ценят консоли, которые занимают не много места и которые легко брать с собой.
- В целом по количеству выручки (в млн.) видно что Американский рынок самый широкий. В Европе игры увлекают людей на меньшие деньги. Можно предположить, что это связано как с демографией, так и с более строгим законодательства в большинстве стран Евросоюза, нежели чем в США и Канаде. Японский рынок сам по себе не велик и имеет специфический национальный уклон(что выражается, например, в любви японцев к портативным приставкам)
- Что касается остальных платформ, то их суммы продаж не превышают продажи Топ-5

### Определите для пользователя каждого региона (NA, EU, JP) самые популярные жанры (топ-5). Поясните разницу.

Теперь рассмотрим варианты с выручкой по жанрам в регионе NA

In [ ]:
(df_after_2013.groupby(['genre'])\
                  .agg({'na_sales':'sum'}).sort_values(by = 'na_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные жанры игр в регионе EU')
                    ) 
plt.show()

In [ ]:
na_genre = df_after_2013.pivot_table(index='genre', values='na_sales', aggfunc='sum').nlargest(5, 'na_sales')
all_na_genre = df_after_2013.pivot_table(index='genre', values='na_sales', aggfunc='sum').nsmallest(5, 'na_sales')
all_na_genre.loc['other_genres'] = all_na_genre.sum()
na_genre = pd.concat([na_genre[:5], all_na_genre[5:]])
na_genre

Теперь рассмотрим варианты с выручкой по жанрам в регионе EU

In [ ]:
(df_after_2013.groupby(['genre'])\
                  .agg({'eu_sales':'sum'}).sort_values(by = 'eu_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные жанры игр в регионе EU')
                    ) 
plt.show()

In [ ]:
eu_genre = df_after_2013.pivot_table(index='genre', values='eu_sales', aggfunc='sum').nlargest(5, 'eu_sales')
all_eu_genre = df_after_2013.pivot_table(index='genre', values='eu_sales', aggfunc='sum').nsmallest(5, 'eu_sales')
all_eu_genre.loc['other_genres'] = all_eu_genre.sum()
eu_genre = pd.concat([eu_genre[:5], all_eu_genre[5:]])
eu_genre

Теперь рассмотрим варианты с выручкой по жанрам в регионе JP

In [ ]:
(df_after_2013.groupby(['genre'])\
                  .agg({'jp_sales':'sum'}).sort_values(by = 'jp_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Популярные жанры игр в регионе JP')
                    ) 
plt.show()

In [ ]:
jp_genre = df_after_2013.pivot_table(index='genre', values='jp_sales', aggfunc='sum')
all_jp_genre = df_after_2013.pivot_table(index='genre', values='jp_sales', aggfunc='sum').nsmallest(5, 'jp_sales')
all_jp_genre.loc['other_genres'] = all_jp_genre.sum()
jp_genre = pd.concat([jp_genre[:5], all_jp_genre[5:]])
jp_genre

In [ ]:
df_after_2013_2 = df_after_2013.pivot_table(index='genre', values='jp_sales', aggfunc='sum').nsmallest(15, 'jp_sales')
df_after_2013_2

**Промежуточные выводы:**

- По жанрам в Северной Америке и Европе наблюдаем очень схожую картину: на вершинах популярности такие жанры, как экшн, шутер и спортивные игры. 
- Первое место по жанрам у Японии занимают ролевые игры. На втором уже идут игры жанра экшн. Топ-3 замыкают Музыкальные игры. А вот шутеры, имеющие поплуярность в Сев. Америке и Европе даже не вошли в топ-5 среди японских пользователей. 
- В жанрах мы видим примерно то же что и с продажами на платформах, Северная Америка является самым обширным рынком из имеющихся у нас данных(т.к. мы не рассматриваем рынки игр например Китая, Юго-Восточной Азиии или стран СНГ). Возможно, Сев. Америка даже задаёт моду в выборе жанров для пользователей из Европы. А Японцы, как и ранее, являются специфическим рынком, к которому нужен принципиально другой подход, нежели чем к Европе и Сев. Америке
- Что касается остальных жанров(не вошедших в топ-5), то их суммы продаж не превышают продажи Топ-5

### Определите для пользователя каждого региона (NA, EU, JP), влияет ли рейтинг ESRB на продажи в отдельном регионе?¶

Для понимания следующего показателя, нужно вообще вспомнить, а какие есть возрастные ограничения используемые ESRB:

- «E» («Everyone») — «Для всех»: Содержание вполне подходит для возрастной категории старше 6 лет; такие игры могут понравиться и взрослым.

- «EC» («Early childhood») — «Для детей младшего возраста»: Игра подходит для детей от 3 лет и старше и не содержит материалов, которые родители могли бы счесть неподходящими. Игры, получившие данный рейтинг, изначально разрабатываются для детей и обычно представляют собой развивающие игры.

- «E10+» («Everyone 10 and older») — «Для всех старше 10 лет»: Возрастная категория — от 10 лет. Проекты с данным рейтингом могут содержать немного больше мультипликационного или мягкого насилия, или несколько откровенные сцены.

- «T» («Teen») — «Подросткам 13—19 лет»: Игра подходит для подростков старше 13 лет. Проекты из данной категории могут содержать насилие, непристойные сцены, грубый юмор, не откровенное сексуальное содержимое или не частое использование ненормативной лексики.

- «M» («Mature») — «От 17 лет»: Материалы игры не подходят для возрастной категории младше 17 лет. Проекты с данным рейтингом могут содержать достаточно жестокое насилие, большое количество крови, непристойные сексуальные сцены или грубую ненормативную лексику.

- «E» («Everyone») - То бишь «Для детей и взрослых».

- «AO» («Adults Only 18+») — «Только для взрослых»: Содержание игры только для взрослых. Продукты из данной категории могут содержать длительные сцены жестокого насилия и/или очень откровенное сексуальное содержимое, а также сцены с обнажением.

Теперь рассмотрим возможность влияния ретинга ESRB на продажи в регионе NA

In [ ]:
(df_after_2013.groupby(['rating'])\
                  .agg({'na_sales':'sum'}).sort_values(by = 'na_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Продажи в регионе NA в зависимсоит от ретинга ESRB')
                    ) 
plt.show()

Теперь рассмотрим возможность влияния ретинга ESRB на продажи в регионе EU

In [ ]:
(df_after_2013.groupby(['rating'])\
                  .agg({'eu_sales':'sum'}).sort_values(by = 'eu_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Продажи в регионе EU в зависимсоит от ретинга ESRB')
                    ) 
plt.show()

Теперь рассмотрим возможность влияния ретинга ESRB на продажи в регионе JP

In [ ]:
(df_after_2013.groupby(['rating'])\
                  .agg({'jp_sales':'sum'}).sort_values(by = 'jp_sales', ascending = False ).head(5)
                  .plot(kind = 'bar', grid = True,
                   figsize = (10,5),title = 'Продажи в регионе JP в зависимсоит от ретинга ESRB')
                    ) 
plt.show()

Промежуточные выводы:

- В рейтингах у Северной Америки и Европы большую часть рынка занимает возрастная категория "M" — "От 17 лет" и "Е" - «Для всех». Видимо наибольшими потребителями игр там являются подростки и молодые люди.
- В Японии же первое место занимает категория «unknown». Это свявзано с тем, что игры для платформ в Японии получают свои внутреннние оценки, и их попросту нет в рейтинге ESRB, который предназначен для маркировки игр в США и Канаде. На втором по имеющимся данным, и первом по статистике ESRB месте, Японцы предпочитают игры категории "Е", то есть, для всех возрастов.

**Общие выводы по портретам пользователей в регионах:**

- По регионам самыми популярными платформами являются остаются PS4, X360 и 3DS. Правда, 3DS входит в этот топ из за своего абсолютного превосходства на рынке игровых платформ для японских пользователей.

- Самыми продаваемыми являются жанры - экшн, шутеры и ролевые игры. Здесь опять же, ролевые игры пользуются колоссальной популярностью у пользователей из Японии.

- Идеальным пользователем для Северной Америки, является подросток или взрослый, играющий на платформах PS4 и XOne в экшн, шутеры и спортивные игры.

- Идеальный портрет европейского пользователя так же состоит из подростков и взрослых, играющих на платформе PS4 в экшн, шутеры и спортивные игры.

- Идеальный японский пользователь играет на консоли Nintendo в ролевые игры и экшн.

## Проверьте гипотезы. Задайте самостоятельно пороговое значение alpha.

In [ ]:
df_nnull_user = df_after_2013[df_after_2013['user_score'] > 0]
df_nnull_critic = df_after_2013[df_after_2013['critic_score'] > 0]

### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

**Нулевая гипотеза:** Средние пользовательские рейтинги платформ Xbox One и PC равны

**Альтернативная гипотеза:** Средние пользовательские рейтинги платформ Xbox One и PC не равны

In [ ]:
alpha = .05

platform_XOne = df_nnull_user[(df_nnull_user['platform'] == 'XOne')]['user_score']
platform_PC = df_nnull_user[(df_nnull_user['platform'] == 'PC')]['user_score']

results = st.ttest_ind(platform_XOne.dropna(), platform_PC.dropna(), equal_var=False)
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Дополнительно взглянем на средние пользовательские рейтинги, чтобы показать не противоречивость нашей гипотезы

In [ ]:
print('cредний рейтинг pc: ', platform_PC.mean())
print('cреднее рейтинг xone: ', platform_XOne.mean())

**Вывод по проверке гипотезы: "Средние пользовательские рейтинги платформ Xbox One и PC одинаковые":**

Не получилось отвергнуть нулевую гипотезу, следовательно можем сказать, что средний рейтинг по двум выборкам по платформам почти не различается. Т.к. гипотеза-то наша двухсторонняя: мы не знаем, в какую сторону рейтинг отличается. Фактически P-значение (англ. P-value) - это вероятность ошибки при отклонении нулевой гипотезы (ошибки первого рода). То есть вероятность ошибки при отклонении нулевой гипотезы: ~55%. К тому же посчитав средний рейтинг для двух платформ, с уверенностью можем сказать: средние рейтинги почти не отличаются. Однако принимать какую-либо гипотезу мы в любом случае не можем, т.к. данные могут лишь не противоречить ей или, наоборот, показывать очень маловероятные результаты.

### Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

**Нулевая гипотеза:** Средние пользовательские рейтинги жанров Action и Sports равны

**Альтернативная гипотеза:** Средние пользовательские рейтинги жанров Action и Sports не равны

In [ ]:
alpha = .05

genre_action = df_nnull_user[(df_nnull_user['genre'] == 'Action')]['user_score']
genre_sport = df_nnull_user[(df_nnull_user['genre'] == 'Sports')]['user_score']

results = st.ttest_ind(genre_action, genre_sport, equal_var=False)
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

In [ ]:
print('Средний пользовательский рейтинг action: ', genre_action.mean())
print('Средний пользовательский рейтинг sports: ', genre_sport.mean())

**Вывод по проверке гипотезы: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные":**

Можем сказать, что средний рейтинг по двум жанрам отличается. Т.к. гипотеза-то наша двухсторонняя: мы не знаем, в какую сторону рейтинг отличается. В данном случае вероятность ошибки при отклонении нулевой гипотезы: ~0%. Опять же, посчитав средний рейтинг для двух жанров, с уверенностью можем сказать: средние рейтинги отличаются. На имеющихся данных, на уровне значимости 5% есть основания отвергнуть нулевую гипотезу о равенстве средних значений пользовательских рейтингов (отзывов) жанров Action и Sports в пользу альтернативной гипотезы

### Поясните как вы сформулировали нулевую и альтернативную гипотезы и какой критерий применили для проверки гипотез и почему.

- Нулевая гипотеза подразумевает знак равенства. Альтернативная гипотеза подразумевает неравенство(≠, <, >, ≤, ≥). В наших двух гипотезах альтернативная была неравенством.
- Обе гипотезы проверялись двусторонним t-тестом с проверкой равенства дисперсий. Двусторонний - потому что нам не важно в большую или меньшую сторону будет отклонение от нулевой гипотезы. Критический уровень значимости поставили 5%, потому что точность здесь была не самым важным приоритетом.

## Напишите общий вывод

<div style="border:solid Chocolate 2px; padding: 40px">

Целью данного исследования было выявить определяющие успешность игры закономерности. Это позволит заказчику сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

**I. С чем мы столкнулись в начале исследования и какие рекомендации можем дать, чтобы в следующий раз было меньше проблемных моментов:**
  
- Поступившие к нам данные имели большое количество пропусков (более 40%). Основными причинами пропусков были:
    1. Наличие в Европе и Японии своих организаций, которые занимаются возрастными рейтингами(таких как PEGI и CERO). По возможности стоит добавить в датасет столбцы с этими рейтингами.
    2. До появления ESRB рейтинги видеоигр задавались самими производителями консолей. Напрмер, Nintendo не оценивала игры, но использовала цензуру для контроля за возрастными последствиями для пользователя. Поэтому у игр на Nintendo-платформе огромное количество пропусков в рейтинге ESRB.

**II. Что мы сделали на этапе предобработки данных:**
    
- Столбцы были преведены к правильному регистру
- В некоторых столбцах данные были преобразованы в подходящий для исследования тип
- Были заполнены пропуски и удалены дубликаты там, где это не отразилось бы на дальнейшем исследовании
- Была заменены в пропусках с рейтингом
- Так же мы создали столбец с суммарными продажами по миру

**III. Какие выводы мы сделали из исследовательского анализа:**
    
- Актульным был выбран период с 2013 года, т.к. данные за более ранние годы имели много пропусков, связанных с тем, что:
    1. В более ранние годы отсуствовала официальная статистика
    2. Многие платформы к 2013 году ушли с рынков
- Новые платформы повлялись примерно каждые пять лет, а старые платформы изчезали примерно через два года после появления новых
- На успех игры на рынке сильнее влияют оценки критиков, чем оценки игроков, но прямой корреляции здесь обнаружить не удалось
- В топ-3 продаваемых жанра входит шутер, спортивные игры и платформеры
- Шутеры, несмотря на относительно небольшое количество игр, являются самым перспективным жанром для продаж

**IV. В результате составления портретов пользователя для каждого из регионов(Северная Америка, Европа, Япония) мы выяснили, что:**
    
- В Северной Америке наибольшее количество игроков подросткового и взрослого возрастов, использующие консоли PS4 и XOne для игры в экшн, шутеры и спортивные игры.
- В Европе аналогичная картина, где подростки и взрослые играют в экшн, шутеры и спортивные игры на PS4 и XOne.
- В Японии чаще всего пользователь играет на консоли Nintendo в ролевые игры и экшн.
    
**V. В результатах проверки гипотез были выведены следующие предположения:**
    
- Средние пользовательские рейтинги платформ Xbox One и PC равны
- Средние пользовательские рейтинги жанров Action и Sports не равны

**Для успешной рекламной кампании на 2017 год в Северной Америке и Европе необходимо ориентироваться на платформы выпущенные после 2013 года, а именно PS4 и Xbox One. Лидерами продаж являются, и скоре всего будут оставаться, игры с жанром шутер и спортивные игры с возрастной категорией "M" (от 17 лет) и "E" (для всех).**

**Рынок Японии значительно отличается от Северной Америки и Европы, поэтому для японских пользователей перспективной платформой будет 3DS, а жанром снискавшим всенародную любовь останется "ролевые игры". Рейтинг ESRB в данном случае учитывать не стоит**